In [1]:
import requests
import json
import csv

In [17]:
# Import required libraries
import requests
import xml.etree.ElementTree as ET
import csv

# URL of the API endpoint
url = 'https://lapi.transitchicago.com/api/1.0/ttarrivals.aspx'

# List of mapid values
mapid_values = ['40260', '40380', '41700', '40680', '40850', '40160', '40040', '40730', '40460']

# Open a CSV file for writing
csv_file = open('loop_data.csv', 'w', newline='', encoding='utf-8')

# Create a CSV writer object to write data into the file
csv_writer = csv.writer(csv_file)

# Write the header row in the CSV file, including the 'MapID' column
csv_writer.writerow(['MapID', 'Route', 'Arrival Time', 'Schedule Fault', 'Delayed', 'Destination'])

# Loop through each mapid value
for mapid in mapid_values:
    # Iterate however many times for each mapid (change the number within the quotations)
    for _ in range(20):
        # Parameters to send with the API request
        params = {
            'mapid': mapid,
            'max': '7', 
            'key': 'ccbbacb6affa4dfdb45f796b2c369773'
        }

        # Send a GET request to the API with the specified URL and parameters
        response = requests.get(url, params=params)

        # Check if the response status code is 200 (OK)
        if response.status_code == 200:
            try:
                # Parse the XML content of the response
                root = ET.fromstring(response.text)
                
                # Find all 'eta' elements in the XML
                eta_elements = root.findall(".//eta")

                # Loop through each 'eta' element and extract data
                for eta in eta_elements:
                    # Extract Route
                    route = eta.find("rt").text
                    
                    # Extract Arrival Time
                    arr_time = eta.find("arrT").text
                    
                    # Extract Schedule Fault
                    flt_bool = eta.find("isFlt").text
                    
                    # Extract Delay
                    dly_bool = eta.find("isDly").text
                    
                    # Extract Station and Destination names
                    station_name = eta.find("staNm").text
                    dest_name = eta.find("destNm").text

                    # Write the extracted data as a row in the CSV file, including the mapid
                    csv_writer.writerow([mapid, route, arr_time, flt_bool, dly_bool, station_name, dest_name])

            # Handle parsing errors
            except ET.ParseError as e:
                print("Error parsing XML:", e)
                print("Response content:", response.text)
        else:
            # Print error information if the request fails
            print("Request error - status code:", response.status_code)
            print("Response content:", response.text)

# Close the CSV file
csv_file.close()